# Artificial neural network

In [1]:
# Import PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, flatten, device
from tqdm import tqdm, trange
from scipy import sparse
import json

In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

## model definition

In [31]:
# https://github.com/khanhnamle1994/MetaRec/blob/b5e36cb579a88b32cdfb728f35f645d76b24ad95/Boltzmann-Machines-Experiments/RBM-CF-PyTorch/rbm.py#L23
# Create the Restricted Boltzmann Machine architecture
class network(nn.Module):
    def __init__(self, input_size, output_size, dropout, l1, l2, l3, activation=F.relu):
        super().__init__()
        
        # use 3 layers and fc layer
        self.activation = activation

        self.dropout = nn.Dropout(dropout)
        if torch.cuda.is_available():
            self.device = device("cuda")

        self.lin1 = nn.Linear(input_size, l1).to(self.device)
        self.lin2 = nn.Linear(l1, l2).to(self.device)
        self.lin3 = nn.Linear(l2, l3).to(self.device)
        self.fc = nn.Linear(l3, output_size).to(self.device)

    def forward(self, x):
        if torch.cuda.is_available():
            x = x.to(self.device)

        x = self.activation(self.lin1(x))
        x = self.activation(self.lin2(x))
        x = self.activation(self.lin3(x))

        x = self.dropout(x)
        output = self.fc(x)
        return output

In [4]:
cuda = torch.device('cuda')

## General Imports

In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

tqdm.pandas() #for progres_apply etc.

 
## Reading in data

In [6]:
def load_interactions(path, n_splits=5):
    """
    load in the interactions_splits.pkl.gz file with our data for the various users
    :param path: path location of the data
    :param n_splits: split in n_split splits
    :return: interactions
    """
    df = pd.read_pickle(os.path.join(os.getcwd(), path))
    df[['interactions', 'train', 'val', 'test']] = df[['interactions', 'train', 'val', 'test']].applymap(lambda x: np.array(x, dtype=np.int32))
    interactions_dict = {}
    for split in trange(n_splits):
        for column in ['train', 'val', 'test']:
            interactions_dict[split, column] = pd.DataFrame({
                'user_id': df['user_id'],
                'steam_id': df['steam_id'],
                'item_id': df[column].apply(lambda x: x[split, 0]),
                'playtime_forever': df[column].apply(lambda x: x[split, 1]),
                'playtime_2weeks': df[column].apply(lambda x: x[split, 2])})
    return interactions_dict

In [7]:
interactions = load_interactions("./data-cleaned/interactions_splits.pkl.gz")
interactions[0, 'train'].head()
games = pd.read_pickle(os.path.join(os.getcwd(), "./data-cleaned/games.pkl.gz"))
games.head()

100%|██████████| 5/5 [00:00<00:00,  8.06it/s]


,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore,users_count
0,Rockstar Games,[Action],Grand Theft Auto: Episodes from Liberty City,Grand Theft Auto: Episodes from Liberty City,http://store.steampowered.com/app/12220/Grand_...,2010-04-12,"[Open World, Action, Third Person, Multiplayer...",NaN,http://steamcommunity.com/app/12220/reviews/?b...,"[Single-player, Multi-player]",19.99,False,12220,Rockstar North / Toronto,Mostly Positive,NaN,7597
1,Valve,[Action],Half-Life,Half-Life,http://store.steampowered.com/app/70/HalfLife/,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",NaN,http://steamcommunity.com/app/70/reviews/?brow...,"[Single-player, Multi-player, Valve Anti-Cheat...",9.99,False,70,Valve,Overwhelmingly Positive,96,7575
2,"Trion Worlds, Inc.","[Action, Free to Play, Massively Multiplayer, ...",Defiance,Defiance,http://store.steampowered.com/app/224600/Defia...,2014-06-04,"[Free to Play, Action, Open World, Massively M...",NaN,http://steamcommunity.com/app/224600/reviews/?...,"[Multi-player, MMO, Co-op, Steam Trading Cards...",Free to Play,False,224600,"Trion Worlds, Inc.",Mostly Positive,64,7539
3,Bohemia Interactive,"[Action, Simulation, Strategy]",Arma 3,Arma 3,http://store.steampowered.com/app/107410/Arma_3/,2013-09-12,"[Simulation, Military, Multiplayer, Realistic,...",NaN,http://steamcommunity.com/app/107410/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",39.99,False,107410,Bohemia Interactive,Very Positive,74,7527
4,Unknown Worlds Entertainment,"[Action, Indie, Strategy]",Natural Selection 2,Natural Selection 2,http://store.steampowered.com/app/4920/Natural...,2012-10-30,"[Multiplayer, Strategy, FPS, Team-Based, Actio...",NaN,http://steamcommunity.com/app/4920/reviews/?br...,"[Multi-player, Online Multi-Player, Steam Achi...",9.99,False,4920,Unknown Worlds Entertainment,Very Positive,80,7502


In [8]:
train0 = interactions[0, 'train']
test0 = interactions[0, 'test']

In [9]:
train0["item_id"].map(len).describe()

count    54190.000000
mean        25.977265
std         42.254444
min          1.000000
25%          5.000000
50%         13.000000
75%         30.000000
max        674.000000
Name: item_id, dtype: float64

In [10]:
train0.iloc[100,:]

user_id                                                 EucHellscythe
steam_id                                            76561198072757340
item_id             [991, 214, 4594, 1262, 30, 3449, 959, 964, 436...
playtime_forever    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
playtime_2weeks     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 100, dtype: object

## Sparse Matrix


In [11]:
def score_playtime(playtime):
    """
    give a game a rating score between 0 and 4 
    :param playtime: the playtime to give a score by
    :return: 0,1,2,3 or 4 based on playtime
    """
    if playtime < 120:
        # less than 2 hrs
        return 1
    elif playtime < 240:
        # less than 4 hrs
        return 2
    elif playtime < 600:
        # less than 10 hrs
        return 3
    elif playtime < 24*60:
        # less than 24 hrs
        return 4
    else:
        return 5

### Method to create Sparse Matrix

In [12]:


#Create scipy csr matrix
def get_sparse_matrix(df):
    """
    generate a sparse matrix of user-game pairs based on our dataframe
    :param df: the dataframe to base the sparse matrix upon
    :return: a sparse matrix of user-game pairs with a score based on playtime
    """
    shape = (df.shape[0], games.shape[0])
    
    user_ids = []
    item_ids = []
    values = []
    for idx, row in df.iterrows():
        items = row['item_id']
        user = idx
        score = row["playtime_forever"] + 2* row["playtime_2weeks"]
        
        # recommended = row['recommended']
        user_ids.extend([user] * len(items))
        item_ids.extend(items)
        values.extend([score_playtime(score[i]) for i in range(len(items))])
    # create csr matrix
    # values = np.ones(len(user_ids))
    matrix = sparse.csr_matrix((values, (user_ids, item_ids)), shape=shape, dtype=np.int32)
    return matrix

In [13]:
# test_matrix = get_sparse_matrix(test0)
# train_matrix = get_sparse_matrix(train0)
# train_matrix


# Evaluation

## HR / Recall / NDCG Function Definitions

### Vanilla Recommendations

In [14]:
def compute_hr(train_matrix, test_matrix, rbm, k=10, batch_size=100):
    """
    compute the various metrics of our model, hr, recall and ndcg
    :param train_matrix: the input wich our user already has
    :param test_matrix: the games we are trying to recommend to each user
    :param rbm: our model used to make recommendations (should here be an ANN)
    :param k: the amount of recommendations we are going to give
    :param batch_size: the batchsize used to work faster
    :return: hitrates, recall, nDCG as an array, use np.average to get value
    """
    hitrates = []
    recall = []
    nDCG = []
    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0] - batch_size, batch_size): # - batch_size, batch_size):
        v = train_matrix[id_user:id_user + batch_size]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user:id_user + batch_size]  # target
        if vt.getnnz() == 0:
            continue

        v = convert_sparse_matrix_to_sparse_tensor(v)
        vt = convert_sparse_matrix_to_sparse_tensor(vt)
        v = v.to_dense()
        vt = vt.to_dense()
        v = v.sub(1)
        vt = vt.sub(1)

        if torch.cuda.is_available():
            vt = vt.cuda()
            v = v.cuda()

        # ground truth
        users, movies = (vt > 1).nonzero(as_tuple=True)

        indices = torch.stack([users, movies])
        shape = (batch_size, train_matrix.shape[1])
        target = torch.sparse.LongTensor(indices, torch.add(vt[vt > 1].flatten(), 1), torch.Size(shape))
        target_dense = target.to_dense()

        target_rating, target_movie = torch.topk(target_dense, k, 1)
        # target_movie[target_rating < 3] = -1 # remove all bad movies from top k

        values, _ = torch.max(target_rating, dim=1)
        users_with_target = (values > 0).nonzero(as_tuple=True)[0].cpu().tolist()


        # predicted
        # _, h = rbm.sample_h(v)
        # recommended, _ = rbm.sample_v(h)
        recommended = rbm(v)
        recommended[v != -1] = -10
        predicted_rating, predicted_movie = torch.topk(recommended, k)

        # TODO optimize range s.t. users without target are skipped
        for user in users_with_target:

            # all recommendations
            user_target = target_movie[user][target_rating[user] > 0].cpu().tolist()
            user_pred = predicted_movie[user].cpu().tolist()

            counter = 0
            total = min(k, len(user_target))
            for target in user_target:
                if target in user_pred:
                    counter += 1
            # counter = len(recommendations)

            recall.append(counter / total)
            hitrates.append(min(1, counter))

            # nDCG
            idcg = np.sum([1 / np.log2(i+2) for i in range(min(k, len(user_target)))])
            dcg = 0
            for i, r in enumerate(user_pred):
                if r in user_target:
                    dcg += 1 / np.log2(i+2)

            nDCG.append(dcg / idcg) 

    return hitrates, recall, nDCG

# Train model

In [36]:
def params_to_str(params):
    """
    turn a models parameters to a string to paste somewhere
    :param params: the list of parameters
    :return: a string
    """
    return "-".join([s.__name__ if callable(s) else str(s) for s in params] )

def score_model(rbm, batch_size, train_matrix, test_matrix):
    """
    calculate an error for the output of our rbm for the unseen (and untrained upon) test-values
    :param rbm: the model for which we test values
    :param batch_size: the batchsize used for training/testing
    :param train_matrix: the original input with which we try to get our test-values
    :param test_matrix: the values we try for our model to acquire based on train_matrix
    :return: the RMSE for our test-values
    """
    test_recon_error = 0  # RMSE reconstruction error initialized to 0 at the beginning of training
    s = 0  # a counter (float type) 
    # for loop - go through every single user
    for id_user in range(0, train_matrix.shape[0] - batch_size, batch_size):
        v = train_matrix[id_user:id_user + batch_size]  # training set inputs are used to activate neurons of my RBM
        vt = test_matrix[id_user:id_user + batch_size]  # target
        v = convert_sparse_matrix_to_sparse_tensor(v)
        vt = convert_sparse_matrix_to_sparse_tensor(vt)

        v = v.to_dense()
        vt = vt.to_dense()
        v = v.sub(1)
        vt = vt.sub(1)
        
        if torch.cuda.is_available():
            v = v.cuda()
            vt = vt.cuda()

        if len(vt[vt > -1]) > 0:
            vk = rbm(v)
            
            # Update test RMSE reconstruction error
            loss = torch.sqrt(torch.mean((vt[vt > -1] - vk[vt > -1])**2))
            loss.backward()
            test_recon_error += loss
            s += 1

    return test_recon_error / s 


# https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(X, k=5):
    """
    turn the Sparse scipy matrix into a sparse pytorch tensor
    :param X: the Sparse scipy matrix
    :param k: the amount of possible ratings we have given to our user-game pairs
    :return: a sparse 3-D pytorch tensor of dimensions game-user-rating
    """
    coo = X.tocoo()

    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    # print(values)
    # print("values", v)
    shape = coo.shape
    tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape)) 
    if torch.cuda.is_available():
        tensor = tensor.cuda()

    return tensor 
    
def create_rbm(train_matrix, test_matrix, batch_size, epochs, params, model=None, k=5, hrmod=20):
    """
    generate and train an ANN on train_matrix as input (use name to be backward compatible)
    :param train_matrix: the input upon which our model is trained
    :param test_matrix: the input upon which our model is validated
    :param batch_size: the batchsize we use
    :param epochs: the amount of epochs we will be running
    :param model: an optional variable that if not None trains a pre-generated model further instead of generating a new one
    :param k: the amount of possible ratings we have given to our user-game pairs
    :return: a trained RBM
    """
    n_vis = train_matrix.shape[1]
    train_errors = []
    test_errors = []
    if model is None:
        model = network(n_vis, n_vis, *params)
    optim = torch.optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

    metrics = {
        "hr": [],
        "r": [],
        "ndcg": []
    }

    print("start training")
    for epoch in trange(epochs):
        model.train()
        train_recon_error = 0  # RMSE reconstruction error initialized to 0 at the beginning of training
        s = 0
        
        for user_id in range(0, train_matrix.shape[0] - batch_size, batch_size):
            training_sample = train_matrix[user_id : user_id + batch_size]
            v0 = convert_sparse_matrix_to_sparse_tensor(training_sample)

            v0 = v0.to_dense()
            v0 = v0.sub(1)
            
            optim.zero_grad()            
            vk = model(v0)
            loss = torch.sqrt(torch.mean((v0[v0 > -1] - vk[v0 > -1])**2))
            loss.backward()
            optim.step()
            train_recon_error +=loss
            s += 1
            
        train_errors.append(train_recon_error / s)

        model.eval()
        test_errors.append(score_model(model, batch_size, train_matrix, test_matrix))

        if epoch % hrmod == hrmod - 1:
            hr, r, ndcg = compute_hr(train_matrix, test_matrix, model, batch_size=batch_size)
            metrics["hr"].append(np.average(hr))
            metrics["r"].append(np.average(r))
            metrics["ndcg"].append(np.average(ndcg))


    import matplotlib.pyplot as plt
    # Plot the RMSE reconstruction error with respect to increasing number of epochs
    plt.plot(torch.Tensor(train_errors, device='cpu'), label="train")
    plt.plot(torch.Tensor(test_errors, device='cpu'), label="test")
    plt.ylabel('Error')
    plt.xlabel('Epoch')
    plt.legend()


    plt.savefig(f'ann-{params_to_str(params)}-{batch_size}-{epochs}.jpg')
    plt.show()
    plt.clf()

    return model, metrics


# Hyperparam Tuning

test various combinations of layer inputs dropout activationfunctions and scoring functions to see which performs best

In [21]:
train_matrix = get_sparse_matrix(train0)
test_matrix = get_sparse_matrix(test0)

In [ ]:
l1s = np.arange(4, 32, 4)
l2s = np.arange(8, 65, 8)
l3s = np.arange(16, 129, 16)
# l1s = [48]
# l2s = [32, 48]
# dropouts = [0.1, 0.2, 0.3]
dropouts = [0.1]

epochs = 100
activation = torch.tanh

for l1 in l1s:
    for l2 in l2s:
        for l3 in l3s:
            for dropout in dropouts:
                model, metrics = create_rbm(train_matrix, test_matrix, 10000, epochs, (dropout, l1, l2, l3, activation), hrmod=20)
                
                torch.save(model.state_dict(), f"./ann-rating-{l1}-{l2}-{l3}-{dropout}-steam{epochs}-train0")
                with open(f"metrics-rating-{l1}-{l2}-{l3}-{dropout}.json", "w") as f:
                    f.write(json.dumps(metrics))